<a href="https://colab.research.google.com/github/JackHidary/quantumcomputingbook/blob/master/Notes/textbook_algos_hands_on_pub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Textbook Algorithms in Cirq

In this notebook we'll run through some Cirq implementations of some of the standard algorithms that one encounters in an introductory quantum computing course. The discussion here is expanded from examples found in the [Cirq examples](https://github.com/quantumlib/Cirq/tree/master/examples) directory.

## Instructions

Go to File --> Save Copy in Drive to get your own copy to play with.



In [0]:
# install cirq
!pip install cirq==0.5 --quiet

     |████████████████████████████████| 716kB 2.7MB/s 
     |████████████████████████████████| 12.8MB 45.0MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


To verify that Cirq is installed in your environment, try to `import cirq` and print out a diagram of the Bristlecone device.

In [0]:
import cirq
import numpy as np
import random

print(cirq.google.Bristlecone)

                                             (0, 5)────(0, 6)
                                             │         │
                                             │         │
                                    (1, 4)───(1, 5)────(1, 6)────(1, 7)
                                    │        │         │         │
                                    │        │         │         │
                           (2, 3)───(2, 4)───(2, 5)────(2, 6)────(2, 7)───(2, 8)
                           │        │        │         │         │        │
                           │        │        │         │         │        │
                  (3, 2)───(3, 3)───(3, 4)───(3, 5)────(3, 6)────(3, 7)───(3, 8)───(3, 9)
                  │        │        │        │         │         │        │        │
                  │        │        │        │         │         │        │        │
         (4, 1)───(4, 2)───(4, 3)───(4, 4)───(4, 5)────(4, 6)────(4, 7)───(4, 8)───(4, 9)───(4, 10)
         │        │      

## Quantum Teleportation

Quantum Teleportation is a process by which a quantum state can be transmitted
by sending only two classical bits of information. This is accomplished by
pre-sharing an entangled state between the sender (Alice) and the receiver
(Bob). This entangled state allows the receiver (Bob) of the two classical
bits of information to possess a qubit with the same state as the one held by
the sender (Alice).
In the following example output, qubit 0 (the Message) is set to a random state
by applying X and Y gates. By sending two classical bits of information after
qubit 0 (the Message) and qubit 1 (Alice's entangled qubit) are measured, the
final state of qubit 2 (Bob's entangled qubit) will be identical to the
original random state of qubit 0 (the Message). This is only possible given
that an entangled state is pre-shared between Alice and Bob.

=== REFERENCES ===

https://en.wikipedia.org/wiki/Quantum_teleportation
https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.70.1895

In [0]:
def make_quantum_teleportation_circuit(gate):
    circuit = cirq.Circuit()
    msg, alice, bob = cirq.LineQubit.range(3)

    # Creates Bell state to be shared between Alice and Bob
    circuit.append([cirq.H(alice), cirq.CNOT(alice, bob)])
    # Creates a random state for the Message
    circuit.append(gate(msg))
    # Bell measurement of the Message and Alice's entangled qubit
    circuit.append([cirq.CNOT(msg, alice), cirq.H(msg)])
    circuit.append(cirq.measure(msg, alice))
    # Uses the two classical bits from the Bell measurement to recover the
    # original quantum Message on Bob's entangled qubit
    circuit.append([cirq.CNOT(alice, bob), cirq.CZ(msg, bob)])

    return circuit


gate = cirq.SingleQubitMatrixGate(cirq.testing.random_unitary(2))
circuit = make_quantum_teleportation_circuit(gate)

print("Circuit:")
print(circuit)

sim = cirq.Simulator()

# Create qubits.
q0 = cirq.LineQubit

# Produces the message using random unitary
message = sim.simulate(cirq.Circuit.from_ops(gate(q0)))

print("Bloch Vector of Message After Random Unitary:")
# Prints the Bloch vector of the Message after the random gate
b0X, b0Y, b0Z = cirq.bloch_vector_from_state_vector(
    message.final_state, 0)
print("x: ", np.around(b0X, 4),
      "y: ", np.around(b0Y, 4),
      "z: ", np.around(b0Z, 4))

# Records the final state of the simulation
final_results = sim.simulate(circuit)

print("Bloch Sphere of Qubit 2 at Final State:")
# Prints the Bloch Sphere of Bob's entangled qubit at the final state
b2X, b2Y, b2Z = cirq.bloch_vector_from_state_vector(
    final_results.final_state, 2)
print("x: ", np.around(b2X, 4),
      "y: ", np.around(b2Y, 4),
      "z: ", np.around(b2Z, 4))

Circuit:
      ┌                         ┐
0: ───│0.23 +0.039j 0.146+0.961j│───────@───H───M───────@───
      │0.405+0.884j 0.171-0.159j│       │       │       │
      └                         ┘       │       │       │
                                        │       │       │
1: ───H─────────────────────────────@───X───────M───@───┼───
                                    │               │   │
2: ─────────────────────────────────X───────────────X───@───
Bloch Vector of Message After Random Unitary:
x:  0.2552 y:  0.3752 z:  -0.8911
Bloch Sphere of Qubit 2 at Final State:
x:  0.2552 y:  0.3752 z:  -0.8911


## Deutsch's Algorithm

Deutsch's algorithm is one of the simplest demonstrations of quantum parallelism
and interference. It takes a black-box oracle implementing a Boolean function
f(x), and determines whether f(0) and f(1) have the same parity using just one
query.  This version of Deutsch's algorithm is a simplified and improved version
from Nielsen and Chuang's textbook.

=== REFERENCE ===

https://en.wikipedia.org/wiki/Deutsch–Jozsa_algorithm
Deutsch, David. "Quantum theory, the Church-Turing Principle and the universal
quantum computer." Proc. R. Soc. Lond. A, 400:97, 1985.

In [0]:
def make_oracle(q0, q1, secret_function):
    """ Gates implementing the secret function f(x)."""

    # coverage: ignore
    if secret_function[0]:
        yield [cirq.CNOT(q0, q1), cirq.X(q1)]

    if secret_function[1]:
        yield cirq.CNOT(q0, q1)


def make_deutsch_circuit(q0, q1, oracle):
    c = cirq.Circuit()

    # Initialize qubits.
    c.append([cirq.X(q1), cirq.H(q1), cirq.H(q0)])

    # Query oracle.
    c.append(oracle)

    # Measure in X basis.
    c.append([cirq.H(q0), cirq.measure(q0, key='result')])
    return c
  
# Choose qubits to use.
q0, q1 = cirq.LineQubit.range(2)

# Pick a secret 2-bit function and create a circuit to query the oracle.
secret_function = [random.randint(0,1) for _ in range(2)]
oracle = make_oracle(q0, q1, secret_function)
print('Secret function:\nf(x) = <{}>'.format(
    ', '.join(str(e) for e in secret_function)))

# Embed the oracle into a quantum circuit querying it exactly once.
circuit = make_deutsch_circuit(q0, q1, oracle)
print('Circuit:')
print(circuit)

# Simulate the circuit.
simulator = cirq.Simulator()
result = simulator.run(circuit)
print('Result of f(0)⊕f(1):')
print(result)

Secret function:
f(x) = <0, 0>
Circuit:
0: ───H───H───M('result')───

1: ───X───H─────────────────
Result of f(0)⊕f(1):
result=0


## Quantum Fourier Transform and Phase Estimation

This section provides an overview of the quantum Fourier transform and its use in the Phase Estimation algorithm in Cirq.

### Quantum Fourier Transform: Overview

We'll start out by reminding ourselves what the [quantum Fourier transform](https://en.wikipedia.org/wiki/Quantum_Fourier_transform) does, and how it should be constructed.

Suppose we have $n$ qubits in the state $|x\rangle$, where $x$ is an integer in the range $0$ to $2^{n-1}$. The quantum Fourier transform (QFT) performs the following operation:
$$
\text{QFT}|x\rangle = \frac{1}{2^{n/2}} \sum_{y=0}^{2^n-1} e^{2\pi i y x/2^n} |y\rangle.
$$
When $x=0$, this is the same as the action of $H^{\otimes n}$ (this is an important sanity check). Though it may not be obvious at first glance, the QFT is actually a unitary transformation. As a matrix, the QFT is given by
$$
\text{QFT} = \begin{bmatrix}
1 & 1 & 1& \cdots &1 \\
1 & \omega & \omega^2& \cdots &\omega^{2^n-1} \\
1 & \omega^2 & \omega^4& \cdots &\omega^{2(2^n-1)}\\
\vdots &\vdots &\vdots &\ddots &\vdots \\
1 &\omega^{2^n-1} &\omega^{2(2^n-1)} &\cdots &\omega^{(2^n-1)(2^n-1)},
\end{bmatrix}
$$
where $\omega = e^{2\pi i /2^n}$. If you believe that the QFT is unitary, then you'll also notice from the matrix form that its inverse is given by a similar expression but with complex-conjugated coefficients:
$$
\text{QFT}^{-1}|x\rangle = \frac{1}{2^{n/2}} \sum_{y=0}^{2^n-1} e^{-2\pi i y x/2^n} |y\rangle.
$$

The construction of the QFT as a circuit follows a simple recursive form, though fully justifying it will take us too far from the main goal of this notebook. We really only need to know what the circuit looks like, and for that we can look at this picture from the Wikipedia article:

>![QFT Circuit](https://upload.wikimedia.org/wikipedia/commons/6/61/Q_fourier_nqubits.png)

We just need to understand the notation a little bit. $x_j$ on the left-hand side represents one of the binary digits of the input $x$. $x_1$ is the most significant bit and $x_n$ the least significant bit:
$$
x = \sum_{j=0}^{n-1} x_{j+1}2^j.
$$
$H$ is the Hadamard gate, as usual. The Controlled-$R_j$ gates are phase gates similar to the Controlled-$Z$ gate. In fact, for us it will be useful to just think of them as fractional powers of Controlled-$Z$ gates:
$$
CR_j = CZ^{\large 1/2^{j-1}}
$$
Finally, on the far right we have the output representing the bts of $y$. The only difference between the left and right side is that the output bits are in a different order: the most significant bit of $y$ is on the bottom and the least significant bit is on the top.

### Quantum Fourier Transform as a Circuit

Let's define a generator which produces the QFT circuit. It should accept a list of qubits as input and `yield`s the gates to construct the QFT in the right order. A useful observation is that the the QFT circuit "repeats" smaller versions of itself as you move from left to right across the diagram.

In [0]:
def make_qft(qubits):
  """Generator for the QFT on an arbitrary number of qubits. With four qubits
  the answer is
  ---H--@-------@--------@---------------------------------------------
        |       |        |
  ------@^0.5---+--------+---------H--@-------@------------------------
                |        |            |       |
  --------------@^0.25---+------------@^0.5---+---------H--@-----------
                         |                    |            |
  -----------------------@^0.125--------------@^0.25-------@^0.5---H---
  """
  # YOUR CODE HERE

#### Solution

In [0]:
def make_qft(qubits):
  """Generator for the QFT on an arbitrary number of qubits. With four qubits
  the answer is
  ---H--@-------@--------@---------------------------------------------
        |       |        |
  ------@^0.5---+--------+---------H--@-------@------------------------
                |        |            |       |
  --------------@^0.25---+------------@^0.5---+---------H--@-----------
                         |                    |            |
  -----------------------@^0.125--------------@^0.25-------@^0.5---H---
  """
  qubits = list(qubits)
  while len(qubits) > 0:
      q_head = qubits.pop(0)
      yield cirq.H(q_head)
      for i, qubit in enumerate(qubits):
          yield (cirq.CZ**(1/2**(i+1)))(qubit, q_head)

In [0]:
num_qubits = 4
qubits = cirq.LineQubit.range(num_qubits)

qft = cirq.Circuit.from_ops(make_qft(qubits))
print(qft)

                  ┌───────┐   ┌────────────┐   ┌───────┐
0: ───H───@────────@───────────@───────────────────────────────────────
          │        │           │
1: ───────@^0.5────┼─────H─────┼──────@─────────@──────────────────────
                   │           │      │         │
2: ────────────────@^0.25──────┼──────@^0.5─────┼─────H────@───────────
                               │                │          │
3: ────────────────────────────@^(1/8)──────────@^0.25─────@^0.5───H───
                  └───────┘   └────────────┘   └───────┘


### Quantum Fourier Transform as a Gate



For later convenience, it will be useful to encapsulate the QFT construction into a single gate. We can inherit from  `cirq.Gate` to define a gate which acts on an unspecified number of qubits, and then use the same strategy as for `make_qft` in the `_decompose_` method of the gate. Fill in the following code block to make a QFT gate.

In [0]:
class QFT(cirq.Gate):
  """Gate for the Quantum Fourier Transformation
  """
  
  def __init__(self, n_qubits):
    self.n_qubits = n_qubits

  def num_qubits(self):
    return self.n_qubits    
    
  def _decompose_(self, qubits):
    # YOUR CODE HERE
            
  # How should the gate look in ASCII diagrams?          
  def _circuit_diagram_info_(self, args):        
    return tuple('QFT{}'.format(i) for i in range(self.n_qubits))

#### Solution

A copy/paste is all that's required here:

In [0]:
class QFT(cirq.Gate):
  """Gate for the Quantum Fourier Transformation
  """
  
  def __init__(self, n_qubits):
    self.n_qubits = n_qubits

  def num_qubits(self):
    return self.n_qubits
    
  def _decompose_(self, qubits):
    """Implements the QFT on an arbitrary number of qubits. The circuit
    for num_qubits = 4 is given by
    ---H--@-------@--------@---------------------------------------------
          |       |        |
    ------@^0.5---+--------+---------H--@-------@------------------------
                  |        |            |       |
    --------------@^0.25---+------------@^0.5---+---------H--@-----------
                           |                    |            |
    -----------------------@^0.125--------------@^0.25-------@^0.5---H---
    """
    qubits = list(qubits)
    while len(qubits) > 0:
        q_head = qubits.pop(0)
        yield cirq.H(q_head)
        for i, qubit in enumerate(qubits):
            yield (cirq.CZ**(1/2**(i+1)))(qubit, q_head)
            
  # How should the gate look in ASCII diagrams?          
  def _circuit_diagram_info_(self, args):        
    return tuple('QFT{}'.format(i) for i in range(self.n_qubits))

#### Test the Circuit

We should confirm that the gate we've defined is actually doing the same thing as the `make_qft` function from before. We can do that with the following test:

In [0]:
num_qubits = 4

qubits = cirq.LineQubit.range(num_qubits)
circuit = cirq.Circuit.from_ops(QFT(num_qubits).on(*qubits))
print(circuit)

qft_test = cirq.Circuit.from_ops(make_qft(qubits))
print(qft_test)
np.testing.assert_allclose(cirq.unitary(qft_test), cirq.unitary(circuit))

0: ───QFT0───
      │
1: ───QFT1───
      │
2: ───QFT2───
      │
3: ───QFT3───
                  ┌───────┐   ┌────────────┐   ┌───────┐
0: ───H───@────────@───────────@───────────────────────────────────────
          │        │           │
1: ───────@^0.5────┼─────H─────┼──────@─────────@──────────────────────
                   │           │      │         │
2: ────────────────@^0.25──────┼──────@^0.5─────┼─────H────@───────────
                               │                │          │
3: ────────────────────────────@^(1/8)──────────@^0.25─────@^0.5───H───
                  └───────┘   └────────────┘   └───────┘


#### Inverse QFT

We also want to implement the inverse QFT, which we'll do with a completely separate gate. Modify the `QFT` gate from above to create a `QFT_inv` gate:

In [0]:
class QFT_inv(cirq.Gate):
  """Gate for the inverse Quantum Fourier Transformation
  """
  
  # YOUR CODE HERE

#### Solution

Compared to the `QFT` code above, we just have to add in a few minus signs. You can convince yourself that this is the same as complex-conjugating the associated unitary matrix of the QFT, which gives us the inverse QFT.

In [0]:
class QFT_inv(cirq.Gate):
  """Gate for the inverse Quantum Fourier Transformation
  """
  
  def __init__(self, n_qubits):
    self.n_qubits = n_qubits

  def num_qubits(self):
    return self.n_qubits   
    
  def _decompose_(self, qubits):
    """Implements the inverse QFT on an arbitrary number of qubits. The circuit
    for num_qubits = 4 is given by
    ---H--@-------@--------@---------------------------------------------
          |       |        |
    ------@^-0.5--+--------+---------H--@-------@------------------------
                  |        |            |       |
    --------------@^-0.25--+------------@^-0.5--+---------H--@-----------
                           |                    |            |
    -----------------------@^-0.125-------------@^-0.25------@^-0.5--H---
    """
    qubits = list(qubits)
    while len(qubits) > 0:
        q_head = qubits.pop(0)
        yield cirq.H(q_head)
        for i, qubit in enumerate(qubits):
            yield (cirq.CZ**(-1/2**(i+1)))(qubit, q_head)
                      
  def _circuit_diagram_info_(self, args):        
    return tuple('QFT{}^-1'.format(i) for i in range(self.n_qubits))

#### Is the QFT_inv gate the inverse of the QFT Circuit?

We chose not to define the `QFT_inv` as literally the inverse of the `QFT` gate. Why was that? What would you get if you concetenated the `QFT` and `QFT_inv` gates as defined? (Try it!) Can you put together `QFT` and `QFT_inv` in such a way that one undoes the action of the other? This exercise doubles as a test of the `QFT_inv` implementation.

#### Solution

The QFT and QFT_inv circuits we have defined are not literal inverses of each other because the both reverse the order of the bits when going from input to output. We can explicitly see this in the following code block:

In [0]:
num_qubits = 2

qubits = cirq.LineQubit.range(num_qubits)
circuit = cirq.Circuit.from_ops(QFT(num_qubits).on(*qubits),
                                QFT_inv(num_qubits).on(*qubits))
print(circuit)
cirq.unitary(circuit).round(2)

0: ───QFT0───QFT0^-1───
      │      │
1: ───QFT1───QFT1^-1───


array([[1. +0.j , 0. +0.j , 0. +0.j , 0. +0.j ],
       [0. +0.j , 0.5+0.5j, 0. +0.j , 0.5-0.5j],
       [0. +0.j , 0.5+0.j , 0.5-0.5j, 0. +0.5j],
       [0. +0.j , 0. -0.5j, 0.5+0.5j, 0.5+0.j ]])

If `QFT` and `QFT_inv` were really inverses then we would have gotten the identity matrix here. There are a couple of ways to fix this. One is to change the implementations of these two gates in such a way that the outputs are "rightside-up." A different solution is to turn the qubits around in between acting with `QFT` and `QFT_inv`. In other words, we can insert the `QFT_inv` gate "upside-down" as follows:

In [0]:
num_qubits = 2

qubits = cirq.LineQubit.range(num_qubits)
circuit = cirq.Circuit.from_ops(QFT(num_qubits).on(*qubits),
                                QFT_inv(num_qubits).on(*qubits[::-1])) # qubit order reversed
print(circuit)
cirq.unitary(circuit)

0: ───QFT0───QFT1^-1───
      │      │
1: ───QFT1───QFT0^-1───


array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

We find the identity matrix, as desired (up to finite-precision numerical errors). Notice that the `QFT_inv` gate is upside-down relative to the `QFT` gate in the diagram. This is why we included the extra digits in the `wire_symobls`!

### Phase Estimation

As an application of our quantum Fourier transform circuit, we'll implement the [phase estimation](https://en.wikipedia.org/wiki/Quantum_phase_estimation_algorithm) algorithm. The phase estimation algorithm uses the inverse QFT to give an estimate of the eigenvalue of a unitary operator. The total circuit that we are going to implement is given by

>![Phase Estimation](https://upload.wikimedia.org/wikipedia/commons/a/a5/PhaseCircuit-crop.svg)

Suppose we have a unitary operator $U$ with eigenvactor $|\psi\rangle$ and eigenvalue $\exp(2\pi i \theta)$. Our objective is to get an $n$-bit approximation to $\theta$. The first step is to construct the state
$$
|\Phi\rangle = \frac{1}{2^{n/2}}\sum_{y=0}^{2^{n-1}} e^{2\pi i y \theta}|y\rangle.
$$
This looks very similar to the output of the QFT applied to the state $|2^n\theta\rangle$, except for the fact that $2^n\theta$ may not be an integer. If $2^n\theta$ *were* an integer, then we would apply the inverse QFT and measure the qubits to read off the binary representation of $2^n\theta$. Even if $2^n\theta$ is not an integer, we can still perform the same procedure and the result will be a sequence of bits that, with high probility, gives an $n$-bit approximation to $\theta$. We just have to repeat the procedure a few times to be sure of the answer.

Since we've already constructed the inverse QFT, all we really have to do is figure out how to construct the magic state $|\Phi\rangle$. This is accomplished by the first part of the circuit picutred above. We begin by applying $H^{\otimes n}$ to the state $|0\rangle$, creating an equal superposition over all basis states:
$$
H^{\otimes n} |0\rangle = \frac{1}{2^{n/2}}\sum_{y=0}^{2^n-1}|y\rangle.
$$
Now we need to insert the correct phase coefficients. This is done by a sequence of Controlled-$U^k$ operations, where the qubits of $y$ are the controls and the $U^k$ operations act on $|\psi \rangle$.

Let's try to implement this part of the procedure in Cirq, and then put it together with our `QFT_inv` from above. For the gate $U$ we'll pick the single-qubit operation
$$
U = Z^{2\theta} = \begin{bmatrix}
1 & 0 \\
0 & e^{2\pi i \theta }
\end{bmatrix}
$$
for $\theta \in [0,1)$. This is just for simplicity and ease of testing. You are invited to write an implementation that accepts an arbitrary $U$.

In [0]:
theta = 0.234 # Try your own
n_bits = 3 # Accuracy of the estimate for theta. Try different values.

qubits = cirq.LineQubit.range(n_bits)
u_bit = cirq.NamedQubit('u')

U = cirq.Z**(2*theta)

phase_estimator = cirq.Circuit()

phase_estimator.append(cirq.H.on_each(*qubits))
for i, bit in enumerate(qubits):
  phase_estimator.append(cirq.ControlledGate(U).on(bit,u_bit)**(2**(n_bits-1-i)))

print(phase_estimator)

0: ───H───@──────────────────────────────
          │
1: ───H───┼──────────@───────────────────
          │          │
2: ───H───┼──────────┼─────────@─────────
          │          │         │
u: ───────Z^-0.128───Z^0.936───Z^0.468───


The next step is to perform the inverse QFT on estimation qubits:

In [0]:
phase_estimator.append(QFT_inv(n_bits).on(*qubits))
print(phase_estimator)

0: ───H───@──────────────────────────────QFT0^-1───
          │                              │
1: ───H───┼──────────@───────────────────QFT1^-1───
          │          │                   │
2: ───H───┼──────────┼─────────@─────────QFT2^-1───
          │          │         │
u: ───────Z^-0.128───Z^0.936───Z^0.468─────────────


At this point we are almost ready to measure the estimation qubits. But we should also specify an initial state for the `u_bit`. By default it will be the state $|0\rangle$, but the phase for that state is trivial with the operator we chose. Inserting a Pauli $X$ operator at the begining of the circuit changes this to the $|1\rangle$ state, which has the nontrivial $\theta$ phase. 

In [0]:
# Add measurements to the end of the circuit
phase_estimator.append(cirq.measure(*qubits, key='m'))

# Add gate to change initial state to |1>
phase_estimator.insert(0,cirq.X(u_bit))

print(phase_estimator)

0: ───H───@──────────────────────────────QFT0^-1───M('m')───
          │                              │         │
1: ───H───┼──────────@───────────────────QFT1^-1───M────────
          │          │                   │         │
2: ───H───┼──────────┼─────────@─────────QFT2^-1───M────────
          │          │         │
u: ───X───Z^-0.128───Z^0.936───Z^0.468──────────────────────


Now we can intstantiate a simulator and make measurements of the estimation qubits. Let the values of these qubits be $a_j$. Then our $n$-bit approximation for $\theta$ is given by
$$
\theta \approx \sum_{j=0}^n a_j2^{-j}.
$$
We'll perform this conversion from bit values to $\theta$-values and then print the results:

In [0]:
sim = cirq.Simulator()
result = sim.run(phase_estimator, repetitions=10)

theta_estimates = np.sum(2**np.arange(n_bits)*result.measurements['m'], axis=1)/2**n_bits
print(theta_estimates)

[0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25]


When `n_bits` is small, we don't get a very accurate estimate. You can go back and increase `n_bits` and rerun the cells if you like.

To make experimentation easier, let's pack all this up into a single function that lets us specify $\theta$, the number of bits of of accuracy we want in our approxmation, and the number of repetitions of the algorithm to perform. You could just copy/paste from the previous cells, but it might be a useful exercise to write the whole thing from scratch without peeking.

In [0]:
def phase_estimation(theta, n_bits, n_reps=10):

  # YOUR CODE HERE
  
  return theta_estimates

#### Solution

Here is a solution that just consists of what we did in previous cells all put together.

In [0]:
def phase_estimation(theta, n_bits, n_reps=10):
  qubits = cirq.LineQubit.range(n_bits)
  u_bit = cirq.NamedQubit('u')

  U = cirq.Z**(2*theta) # Try out a different gate if you like

  phase_estimator = cirq.Circuit()

  phase_estimator.append(cirq.H.on_each(*qubits))
  for i, bit in enumerate(qubits):
    phase_estimator.append(cirq.ControlledGate(U).on(bit,u_bit)**(2**(n_bits-1-i)))
  phase_estimator.append(QFT_inv(n_bits).on(*qubits))
  
  # Measurement gates
  phase_estimator.append(cirq.measure(*qubits, key='m'))

  # Gates to choose initial state for the u_bit. Placing X here chooses the |1> state
  phase_estimator.insert(0,cirq.X(u_bit))

  # Code to simulate measurements
  sim = cirq.Simulator()
  result = sim.run(phase_estimator, repetitions=n_reps)
  
  # Convert measurements into estimates of theta
  theta_estimates = np.sum(2**np.arange(n_bits)*result.measurements['m'], axis=1)/2**n_bits
  
  return theta_estimates

In [0]:
phase_estimation(0.234, 10)

array([0.234375  , 0.23339844, 0.234375  , 0.23339844, 0.234375  ,
       0.23339844, 0.234375  , 0.234375  , 0.23339844, 0.234375  ])

#### Phase Estimation Without an Eigenstate

We can modify our `phase_estimation` method to explore other interesting aspects of the algorithm. Suppose the input to the circuit was not an eigenstate of $U$ at all? You could modify the case we analyzed above by putting an arbitrary single-qubit state on the `u_bit`, or you could replace $Z^{2\theta}$ with something like $X^{2\theta}$ while still passing in a computational basis state. What is the result?

#### Solution

Suppose the operator $U$ has eigenstate $|u_j\rangle$ with eigenvalues $e^{2\pi i \theta_j}$. Then in general if we pass in a state of the form
$$
\sum_j \alpha_j|u_j\rangle
$$
then each time we run the circuit we will get an $n$-bit estimate of one of the $\theta_j$ chosen at random, and the probability of choosing a particular $\theta_j$ is given by $|\alpha_j|^2$. One simple test of this is to modify our above code to pass the state
$$
\frac{|0\rangle + |1\rangle}{\sqrt{2}}
$$
into the phase estimator for $Z^{2\theta}$. The state $|1\rangle$ has eigenvalue $e^{2\pi i \theta_j}$ while the state $|0\rangle$ has eigenvalue $1$.

In [0]:
def phase_estimation(theta, n_bits, n_reps=10):
  qubits = cirq.LineQubit.range(n_bits)
  u_bit = cirq.NamedQubit('u')

  U = cirq.Z**(2*theta)

  phase_estimator = cirq.Circuit()

  phase_estimator.append(cirq.H.on_each(*qubits))
  for i, bit in enumerate(qubits):
    phase_estimator.append(cirq.ControlledGate(U).on(bit,u_bit)**(2**(n_bits-1-i))) # Could have used CZ in this example
  phase_estimator.append(QFT_inv(n_bits).on(*qubits))
  
  phase_estimator.append(cirq.measure(*qubits, key='m'))

  # Changed the X gate here to an H
  phase_estimator.insert(0,cirq.H(u_bit))

  sim = cirq.Simulator()
  result = sim.run(phase_estimator, repetitions=n_reps)
    
  theta_estimates = np.sum(2**np.arange(n_bits)*result.measurements['m'], axis=1)/2**n_bits
  
  return theta_estimates

In [0]:
phase_estimation(0.234,10)

array([0.234375, 0.234375, 0.234375, 0.234375, 0.      , 0.      ,
       0.234375, 0.      , 0.234375, 0.234375])

Notice that half of the measurements yielded the estimate $0$, which corresponds to the eigenvaule $1$. If you experiment with different gates and input states you should see the more general behavior.

Often we won't be able to prepare an exact eigenstate of the operator $U$ we are interested in, so it's very useful to know about this feature of phase estimation. This is crucial for understanding [Shor's algorithm](https://en.wikipedia.org/wiki/Shor%27s_algorithm), for instance.

### Exercise: Quantum Fourier Transform with Unreversed Output

It may be convenient to have an alternative form of the QFT where the output bits are not in the opposite order of the input bits. Can you make an alternative implementation which does that automatically? You may want to consider using SWAP gates.

### Exercise: Phase Estimation with Arbitrary $U$

Try to implement the phase estimation algorithm in a way that an arbitrary gate $U$ can be supplied and tested. After you've done that, you can test the algorithm on some of your favorite two- or three-qubit gates.

### Exercise: QFT and Phase Estimation with Adjacency Constraints

Often on a real machine we can't execute two-qubit gates between qubits that are not right next to each other. You'll have noticed that the circuits we defined above involves connections between many different pairs of qubits, which will likely not all be near each other when we try to run the circuit on an actual chip. See if you can modify the examples we went through above in such a way that Cirq validates them for use on the Foxtail or Bristlecone chips. How bit of an example can you handle?

## Grover's Algorithm

The Grover algorithm takes a black-box oracle implementing a function
$f(x) = 1\text{ if }x=x',~ f(x) = 0 \text{ if } x\neq x'$ and finds $x'$ within a randomly ordered sequence of $N$ items using $O(\sqrt{N}$) operations and $O(N\,  \text{log}N$) gates,
with the probability $p \geq 2/3$.
At the moment, only 2-bit sequences (for which one pass through Grover operator
is enough) are considered.

=== REFERENCE ===

Coles, Eidenbenz et al. Quantum Algorithm Implementations for Beginners
https://arxiv.org/abs/1804.03719

In [0]:
def set_io_qubits(qubit_count):
    """Add the specified number of input and output qubits."""
    input_qubits = [cirq.GridQubit(i, 0) for i in range(qubit_count)]
    output_qubit = cirq.GridQubit(qubit_count, 0)
    return (input_qubits, output_qubit)


def make_oracle(input_qubits, output_qubit, x_bits):
    """Implement function {f(x) = 1 if x==x', f(x) = 0 if x!= x'}."""
    # Make oracle.
    # for (1, 1) it's just a Toffoli gate
    # otherwise negate the zero-bits.
    yield(cirq.X(q) for (q, bit) in zip(input_qubits, x_bits) if not bit)
    yield(cirq.TOFFOLI(input_qubits[0], input_qubits[1], output_qubit))
    yield(cirq.X(q) for (q, bit) in zip(input_qubits, x_bits) if not bit)


def make_grover_circuit(input_qubits, output_qubit, oracle):
    """Find the value recognized by the oracle in sqrt(N) attempts."""
    # For 2 input qubits, that means using Grover operator only once.
    c = cirq.Circuit()

    # Initialize qubits.
    c.append([
        cirq.X(output_qubit),
        cirq.H(output_qubit),
        cirq.H.on_each(*input_qubits),
    ])

    # Query oracle.
    c.append(oracle)

    # Construct Grover operator.
    c.append(cirq.H.on_each(*input_qubits))
    c.append(cirq.X.on_each(*input_qubits))
    c.append(cirq.H.on(input_qubits[1]))
    c.append(cirq.CNOT(input_qubits[0], input_qubits[1]))
    c.append(cirq.H.on(input_qubits[1]))
    c.append(cirq.X.on_each(*input_qubits))
    c.append(cirq.H.on_each(*input_qubits))

    # Measure the result.
    c.append(cirq.measure(*input_qubits, key='result'))

    return c


def bitstring(bits):
    return ''.join(str(int(b)) for b in bits)


qubit_count = 2
circuit_sample_count = 10

#Set up input and output qubits.
(input_qubits, output_qubit) = set_io_qubits(qubit_count)

#Choose the x' and make an oracle which can recognize it.
x_bits = [random.randint(0, 1) for _ in range(qubit_count)]
print('Secret bit sequence: {}'.format(x_bits))

# Make oracle (black box)
oracle = make_oracle(input_qubits, output_qubit, x_bits)

# Embed the oracle into a quantum circuit implementing Grover's algorithm.
circuit = make_grover_circuit(input_qubits, output_qubit, oracle)
print('Circuit:')
print(circuit)

# Sample from the circuit a couple times.
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=circuit_sample_count)

frequencies = result.histogram(key='result', fold_func=bitstring)
print('Sampled results:\n{}'.format(frequencies))

# Check if we actually found the secret value.
most_common_bitstring = frequencies.most_common(1)[0][0]
print('Most common bitstring: {}'.format(most_common_bitstring))
print('Found a match: {}'.format(
    most_common_bitstring == bitstring(x_bits)))


Secret bit sequence: [0, 1]
Circuit:
(0, 0): ───H───X───@───X───H───X───@───X───H───────M('result')───
                   │               │               │
(1, 0): ───H───────@───H───X───H───X───H───X───H───M─────────────
                   │
(2, 0): ───X───H───X─────────────────────────────────────────────
Sampled results:
Counter({'01': 10})
Most common bitstring: 01
Found a match: True
